## Start configuration

In [3]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT_ID

In [15]:
from googleapiclient.discovery import build
from IPython.display import Image
from IPython.core.display import HTML

# E-Commerce demo

### Nettoyer les données d'un site e-commerce. Les photos ne correspondent pas aux annonces 

### Récupération des données dans BigQuery

In [5]:
%%bq query -n ecomerce
SELECT description, tags, pictures FROM `sfeir-bucket.onBoardDemo.ad`

In [6]:
dataCommerce = ecomerce.execute(output_options=bq.QueryOutput.dataframe()).result()
dataCommerce

Exception: accessDenied: Access Denied: Table sfeir-bucket:onBoardDemo.ad: The user 881499317005-compute@developer.gserviceaccount.com does not have permission to query table sfeir-bucket:onBoardDemo.ad.

In [6]:
adNumber = 0
description = dataCommerce.description[adNumber]
tags = dataCommerce.tags[adNumber].split(',')
pictures = dataCommerce.pictures[adNumber].split(',')

In [7]:
print("Description:\n {}".format(description))
print("Tags:\n {}".format(tags))

Description:
 Inspired by road trips and bikers the Black coat embellished with embroidery on the back. The striped lining ensures a cosy look for the winter.
Fabric: 100% cotton 
Lining: 33% acrylic, 22% wool, 15% polyester, 10% alpaca, 10% polyamide, 10% mohair 
Trim: 100% fox fur 
Colour: Black/Army Black 
Made in Italy
Tags:
 ['trench coat', 'black', 'luxury']


<img src="https://storage.googleapis.com/onboard-demo/handbag-images/coat_1.jpeg" />
<img src="https://storage.googleapis.com/onboard-demo/handbag-images/coat_2.jpeg" />
<img src="https://storage.googleapis.com/onboard-demo/handbag-images/coat_3.jpeg" />



In [8]:
# Running Vision API to find images that have a specific search term
import base64

for IMAGE in pictures:
  print (IMAGE)
  vservice = build('vision', 'v1', developerKey=APIKEY)
  request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'LABEL_DETECTION',
                    'maxResults': 10,
                }]
            }],
        })
  outputs = request.execute(num_retries=3)
  #print (outputs)
  if 'responses' in outputs and len(outputs['responses']) > 0 and 'labelAnnotations' in outputs['responses'][0]:
    for output in outputs['responses'][0]['labelAnnotations']:
      for term in tags:
        if term in output['description']:
          print (u"image={} contains : {} with score {}".format(IMAGE, output['description'],output['score']))

gs://onboard-demo/handbag-images/coat_1.jpeg
image=gs://onboard-demo/handbag-images/coat_1.jpeg contains : black with score 0.9573391
image=gs://onboard-demo/handbag-images/coat_1.jpeg contains : trench coat with score 0.80833113
gs://onboard-demo/handbag-images/coat_2.jpeg
image=gs://onboard-demo/handbag-images/coat_2.jpeg contains : trench coat with score 0.862021
gs://onboard-demo/handbag-images/coat_3.jpeg


<h2> Finding specific text in a corpus of scanned documents </h2>

In [9]:
import subprocess
images = ["https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot1.png",
"https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot2.png",
"https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot5.png"]
print (images)

['https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot1.png', 'https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot2.png', 'https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot5.png']


Here are a few of the images we are going to search.

<img src="https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot1.png" />
<img src="https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot2.png" />
<img src="https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot5.png" />

In [21]:
# Running Vision API to find images that have a specific search term
import base64

SEARCH_TERM = u"1321"

for IMAGE in images:
  #print (IMAGE)
  vservice = build('vision', 'v1', developerKey=APIKEY)
  request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 100,
                }]
            }],
        })
  outputs = request.execute(num_retries=3)
  #print(outputs)
  if 'responses' in outputs and len(outputs['responses']) > 0 and 'textAnnotations' in outputs['responses'][0]:
    for output in outputs['responses'][0]['textAnnotations']:
       if SEARCH_TERM in output['description']:
          print (u"image={} contains the following text: {}".format(IMAGE,SEARCH_TERM))


image=https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot2.png contains the following text: 1321
<img src="https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot2.png" />
image=https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot2.png contains the following text: 1321
<img src="https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot2.png" />


<img src="https://storage.googleapis.com/cloud-training-demos-ml/unstructured/photos/snapshot2.png" />
